In [63]:
import tensorflow as tf
import numpy as np
from keras.utils import image_dataset_from_directory

import os
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import math
from PIL import Image


tf.random.set_seed(0)
np.random.seed(0)

In [64]:
train_ds = image_dataset_from_directory(
    directory='../datasets/train',
    labels='inferred',
    label_mode='categorical',
    batch_size=64,
    # shuffle=True,
    image_size=(48, 48))
validation_ds = image_dataset_from_directory(
    directory='../datasets/validation/',
    labels='inferred',
    label_mode='categorical',
    batch_size=64,
    image_size=(48, 48))

Found 28821 files belonging to 7 classes.
Found 7066 files belonging to 7 classes.


### GRAYSCALE + NORMALIZATION

In [65]:
def normalize(x, y):
    array = tf.image.rgb_to_grayscale(x)
    array = tf.cast(array, tf.float32) / 255.0
    return array, y

In [66]:
def undo_normalization(x, y):
    return tf.cast(x*255, tf.int32), y

In [67]:
train_ds = train_ds.map(normalize)
validation_ds = validation_ds.map(normalize)

# next(train_ds.as_numpy_iterator())
# next(validation_ds.as_numpy_iterator())

### NOISING

In [32]:
def noise(x, y):
    noise_factor = 0.05
    max_pixel_val = 1.0

    noisy_array = x + noise_factor * tf.random.normal(shape=tf.shape(x), mean=0.0, stddev=1.0, dtype=tf.float32) 
    return tf.clip_by_value(noisy_array, 0.0, max_pixel_val), y

In [33]:
dataset1 = tf.data.Dataset.from_tensor_slices(next(train_ds.as_numpy_iterator()))


for img, label in train_ds:
    if tf.reduce_all(tf.equal(label, tf.constant([0,1,0,0,0,0,0], dtype=tf.float32))):
        dataset2 = tf.data.Dataset.from_tensor_slices((img, label))
        dataset1 = dataset1.concatenate(dataset2.map(noise))


# for _ in range(8):
train_ds = train_ds.concatenate(dataset1)

train_ds.shuffle(buffer_size=4)

train_ds = train_ds.map(noise)

itera = dataset1.as_numpy_iterator()
print(next(itera))
next(itera)

(array([[[0.2705612 ],
        [0.29800943],
        [0.30977294],
        ...,
        [0.19998   ],
        [0.16468942],
        [0.1529259 ]],

       [[0.24703412],
        [0.23919177],
        [0.22350706],
        ...,
        [0.21174352],
        [0.19998   ],
        [0.17253175]],

       [[0.18037412],
        [0.15684707],
        [0.22742824],
        ...,
        [0.18037412],
        [0.21174352],
        [0.18821648]],

       ...,

       [[0.35682705],
        [0.2509553 ],
        [0.17645295],
        ...,
        [0.2823247 ],
        [0.20782235],
        [0.16468942]],

       [[0.29408827],
        [0.30977294],
        [0.29800943],
        ...,
        [0.4195659 ],
        [0.26664   ],
        [0.17645295]],

       [[0.3019306 ],
        [0.29016706],
        [0.3176153 ],
        ...,
        [0.40780237],
        [0.4195659 ],
        [0.3176153 ]]], dtype=float32), array([1., 0., 0., 0., 0., 0., 0.], dtype=float32))


(array([[[0.19441243],
         [0.32815516],
         [0.05135371],
         ...,
         [0.10585976],
         [0.16472706],
         [0.1942186 ]],
 
        [[0.22335203],
         [0.27535138],
         [0.13101582],
         ...,
         [0.08779611],
         [0.07661747],
         [0.16605446]],
 
        [[0.13195796],
         [0.2569128 ],
         [0.20931114],
         ...,
         [0.13102673],
         [0.12377119],
         [0.1329372 ]],
 
        ...,
 
        [[0.5477411 ],
         [0.6891414 ],
         [0.7930833 ],
         ...,
         [0.7737834 ],
         [0.8953987 ],
         [0.7991499 ]],
 
        [[0.55071795],
         [0.7051073 ],
         [0.8697194 ],
         ...,
         [0.5764028 ],
         [0.8399145 ],
         [0.797101  ]],
 
        [[0.4148757 ],
         [0.80137765],
         [0.83145374],
         ...,
         [0.5986161 ],
         [0.8685306 ],
         [0.7554033 ]]], dtype=float32),
 array([0., 1., 0., 0., 0., 0., 0.], dty

### Upscaling

In [34]:
def upsample(x, y):
    print()
    return tf.image.resize(x, [150, 150]), y

In [35]:
train_ds = train_ds.map(upsample)
validation_ds = validation_ds.map(upsample)

next(train_ds.as_numpy_iterator())
# next(validation_ds.as_numpy_iterator())

ValueError: in user code:

    File "C:\Users\Emeze\AppData\Local\Temp\ipykernel_43396\725284118.py", line 3, in upsample  *
        return tf.image.resize(x, [150, 150]), y

    ValueError: 'images' contains no shape.


### SAVING DATASET

In [68]:
train_ds.save('../datasets/augmented_train_ds')
validation_ds.save('../datasets/augmented_validation_ds')

### FACE ALIGNER

In [21]:
def euclidean_distance(a, b):
	x1 = a[0]; y1 = a[1]
	x2 = b[0]; y2 = b[1]
	
	return math.sqrt(((x2 - x1) * (x2 - x1)) + ((y2 - y1) * (y2 - y1)))

def detectFace(img):
	faces = face_detector.detectMultiScale(img, 1.3, 5)
	#print("found faces: ", len(faces))

	if len(faces) > 0:
		face = faces[0]
		face_x, face_y, face_w, face_h = face
		img = img[int(face_y):int(face_y+face_h), int(face_x):int(face_x+face_w)]
		img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
		
		return img, img_gray
	else:
		img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
		return img, img_gray
		#raise ValueError("No face found in the passed image ")

def alignFace(img_arr):
	# img = cv2.imread(img_path)
	# plt.imshow(img[:, :, ::-1])
	# plt.show()
    pil_image = Image.fromarray(img_arr).convert('RGB') 
    open_cv_image = np.array(pil_image) 
    # Convert RGB to BGR 
    img = open_cv_image[:, :, ::-1].copy() 

    img_raw = img.copy()

    img, gray_img = detectFace(img)

    global eye_detector
    eyes = eye_detector.detectMultiScale(gray_img)
	
	#print("found eyes: ",len(eyes))

    if len(eyes) >= 2:
        #find the largest 2 eye
        
        base_eyes = eyes[:, 2]
        #print(base_eyes)
        
        items = []
        for i in range(0, len(base_eyes)):
            item = (base_eyes[i], i)
            items.append(item)
        
        df = pd.DataFrame(items, columns = ["length", "idx"]).sort_values(by=['length'], ascending=False)
        
        eyes = eyes[df.idx.values[0:2]]
        
        #--------------------
        #decide left and right eye
        
        eye_1 = eyes[0]; eye_2 = eyes[1]
        
        if eye_1[0] < eye_2[0]:
            left_eye = eye_1
            right_eye = eye_2
        else:
            left_eye = eye_2
            right_eye = eye_1
        
        #--------------------
        #center of eyes
        
        left_eye_center = (int(left_eye[0] + (left_eye[2] / 2)), int(left_eye[1] + (left_eye[3] / 2)))
        left_eye_x = left_eye_center[0]; left_eye_y = left_eye_center[1]
        
        right_eye_center = (int(right_eye[0] + (right_eye[2]/2)), int(right_eye[1] + (right_eye[3]/2)))
        right_eye_x = right_eye_center[0]; right_eye_y = right_eye_center[1]
        
        #----------------------
        #find rotation direction
        
        if left_eye_y > right_eye_y:
            point_3rd = (right_eye_x, left_eye_y)
            direction = -1 #rotate same direction to clock
        else:
            point_3rd = (left_eye_x, right_eye_y)
            direction = 1 #rotate inverse direction of clock
        
        #----------------------
        
        cv2.circle(img, point_3rd, 2, (255, 0, 0) , 2)
        
        cv2.line(img,right_eye_center, left_eye_center,(67,67,67),1)
        cv2.line(img,left_eye_center, point_3rd,(67,67,67),1)
        cv2.line(img,right_eye_center, point_3rd,(67,67,67),1)
        
        a = euclidean_distance(left_eye_center, point_3rd)
        b = euclidean_distance(right_eye_center, point_3rd)
        c = euclidean_distance(right_eye_center, left_eye_center)
        
        cos_a = (b*b + c*c - a*a)/(2*b*c)
        angle = np.arccos(cos_a)
        
        angle = (angle * 180) / math.pi
        
        if direction == -1:
            angle = 90 - angle
        
        new_img = Image.fromarray(img_raw)
        new_img = np.array(new_img.rotate(direction * angle))
        print(angle)

    return new_img

In [28]:
def align_tf(image, label):
    im_shape = image.shape
    image = tf.numpy_function(alignFace, [image], [tf.float32])
    print(image[0])
    image.set_shape(im_shape)
    return image, label

In [29]:
opencv_home = cv2.__file__
folders = opencv_home.split(os.path.sep)[0:-1]

path = folders[0]
for folder in folders[1:]:
	path = path + "/" + folder

face_detector_path = path+"/data/haarcascade_frontalface_default.xml"
eye_detector_path = path+"/data/haarcascade_eye.xml"
nose_detector_path = path+"/data/haarcascade_mcs_nose.xml"


if os.path.isfile(face_detector_path) != True:
	raise ValueError("Confirm that opencv is installed on your environment! Expected path ",detector_path," violated.")

face_detector = cv2.CascadeClassifier(face_detector_path)
eye_detector = cv2.CascadeClassifier(eye_detector_path) 
nose_detector = cv2.CascadeClassifier(nose_detector_path) 

In [30]:
train_ds = train_ds.map(align_tf)
next(train_ds.as_numpy_iterator())

Tensor("PyFunc:0", dtype=float32, device=/job:localhost/replica:0/task:0)


AttributeError: in user code:

    File "C:\Users\Emeze\AppData\Local\Temp\ipykernel_16964\1378897401.py", line 5, in align_tf  *
        image.set_shape(im_shape)

    AttributeError: 'list' object has no attribute 'set_shape'


### Normalizer

In [ ]:
import dlib
def normalize_face(x, y):
    #http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
    face_detector = dlib.get_frontal_face_detector()
    landmark_detector = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
    img = dlib.load_rgb_image("aligned.jpg")
    base_img = img.copy()
    plt.imshow(img)

    faces = face_detector(img, 1)
    faces

    img = base_img.copy()
    landmark_tuple = []
    for k, d in enumerate(faces):
        print("Detection {}: Left: {} Top: {} Right: {} Bottom: {}".format(
                k, d.left(), d.top(), d.right(), d.bottom()))
        
        landmarks = landmark_detector(img, d)
        #print(landmarks)
        
        for n in range(0, 27):
            x = landmarks.part(n).x 
            y = landmarks.part(n).y 
            landmark_tuple.append((x, y))
            
            cv2.circle(img, (x, y), 2, (255, 255, 0), -1)
            

    routes = []

    for i in range(15, -1, -1):
        from_coordinate = landmark_tuple[i+1]
        to_coordinate = landmark_tuple[i]
        routes.append(from_coordinate)

    from_coordinate = landmark_tuple[0]
    to_coordinate = landmark_tuple[17]
    routes.append(from_coordinate)

    for i in range(17, 20):
        from_coordinate = landmark_tuple[i]
        to_coordinate = landmark_tuple[i+1]    
        routes.append(from_coordinate)

    from_coordinate = landmark_tuple[19]
    to_coordinate = landmark_tuple[24]
    routes.append(from_coordinate)

    for i in range(24, 26):
        from_coordinate = landmark_tuple[i]
        to_coordinate = landmark_tuple[i+1]    
        routes.append(from_coordinate)

    #---------------------------------------

    from_coordinate = landmark_tuple[26]
    to_coordinate = landmark_tuple[16]
    routes.append(from_coordinate)
    routes.append(to_coordinate)

    img = base_img.copy()
    for i in range(0, len(routes)-1):
        from_coordinate = routes[i]
        to_coordinate = routes[i+1]
        
        #print(from_coordinate," to ",to_coordinate)
        
        # Showing the mask outline to the face
        # img = cv2.line(img, from_coordinate, to_coordinate, (255, 255, 0), 1)
    plt.figure(figsize = (7,7))
    plt.imshow(img)

    mask = np.zeros((img.shape[0], img.shape[1]))
    mask = cv2.fillConvexPoly(mask, np.array(routes), 1)
    mask = mask.astype(bool)
    #plt.imshow(mask)

    out = np.zeros_like(img)
    out[mask] = img[mask]
    plt.figure(figsize = (7,7))
    plt.imshow(out)

    # save the image
    cv2.imwrite('normalized.jpg', out)